In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import time
%matplotlib inline

In [2]:
ftypes_train = dict(row_id=np.int32,x=np.float32,y=np.float32,accuracy=np.int16,time=np.int32,place_id=np.int64,group=np.int8)
#ftypes_test = dict(row_id=np.int32, x=np.float32, y=np.float32, accuracy=np.int32, time=np.int32)
df_train = pd.read_csv("train_with_place_group.csv", dtype=ftypes_train)
#df_test = pd.read_csv("test.csv", dtype=ftypes_test)

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29118021 entries, 0 to 29118020
Data columns (total 7 columns):
row_id      int32
x           float32
y           float32
accuracy    int16
time        int32
place_id    int64
group       int8
dtypes: float32(2), int16(1), int32(2), int64(1), int8(1)
memory usage: 749.8 MB


In [4]:
df_train.group.value_counts()

0    14559478
1    14558543
Name: group, dtype: int64

In [5]:
df_train.head()

,row_id,x,y,accuracy,time,place_id,group
0,0,0.7941,9.0809,54,470702,8523065625,0
1,1,5.9567,4.7968,13,186555,1757726713,0
2,2,8.3078,7.0407,74,322648,1137537235,1
3,3,7.3665,2.5165,65,704587,6567393236,1
4,4,4.0961,1.1307,31,472130,7440663949,1


In [6]:
print('    Creating datetime features ...')
##time related features (assuming the time = minutes)
initial_date = np.datetime64('2014-01-01T01:01',   #Arbitrary decision
                             dtype='datetime64[m]') 
    #working on df_train  
d_times = pd.DatetimeIndex(initial_date + np.timedelta64(int(mn), 'm') 
                           for mn in df_train.time.values)    
df_train['hour'] = d_times.hour
df_train[['hour']] = df_train[['hour']].astype(np.int16)
df_train['weekday'] = d_times.weekday
df_train[['weekday']] = df_train[['weekday']].astype(np.int16)
df_train['day'] = d_times.day
df_train[['day']] = df_train[['day']].astype(np.int16)
df_train['month'] = d_times.month
df_train[['month']] = df_train[['month']].astype(np.int16)
#df_train['year'] = d_times.year
df_train = df_train.drop(['time'], axis=1)

    Creating datetime features ...


In [ ]:
'''
df_train[['x', 'y']] = df_train[['x', 'y']].astype(np.float32)
df_train[['row_id']] = df_train[['row_id']].astype(np.int32)
df_train[['place_id']] = df_train[['place_id']].astype(np.int64)
df_train[['group']] = df_train[['group']].astype(np.int8)
df_train[['hour','weekday','day','month','accuracy']] = df_train[['hour','weekday','day','month','accuracy']].astype(np.int16)
'''

In [7]:
df_train.head()

,row_id,x,y,accuracy,place_id,group,hour,weekday,day,month
0,0,0.7941,9.0809,54,8523065625,0,3,0,24,11
1,1,5.9567,4.7968,13,1757726713,0,19,5,10,5
2,2,8.3078,7.0407,74,1137537235,1,7,2,13,8
3,3,7.3665,2.5165,65,6567393236,1,13,1,5,5
4,4,4.0961,1.1307,31,7440663949,1,2,1,25,11


In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29118021 entries, 0 to 29118020
Data columns (total 10 columns):
row_id      int32
x           float32
y           float32
accuracy    int16
place_id    int64
group       int8
hour        int16
weekday     int16
day         int16
month       int16
dtypes: float32(2), int16(5), int32(1), int64(1), int8(1)
memory usage: 860.8 MB


In [ ]:
df_train.to_csv('train_xgboost.csv',index=False)